In [15]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [9]:
# Starting with 10k observations
df0 = pd.read_csv('sampled_data_50k.csv')
df0 = df0.sample(n=10000, random_state=2023)

In [25]:
df1 = pd.read_csv('sampled_data_50k.csv')
df1['action_taken'].value_counts

<bound method IndexOpsMixin.value_counts of 0        3
1        1
2        3
3        1
4        5
        ..
49995    1
49996    1
49997    5
49998    1
49999    1
Name: action_taken, Length: 50000, dtype: int64>

In [10]:
# Dropping denial reason because it perfectly predicts approval or denial 
df = df0.drop(columns=['denial_reason-1', 'denial_reason-2', 'denial_reason-3', 'denial_reason-4'])

### Filling in NA values

In [11]:
# Some columns are being stored as strings because they have an 'exempt' value.

# Finding columns with exempt
columns_with_exempt = []

# Iterate through all columns
for col in df.columns:
    # Check if 'Exempt' is in the column
    if (df[col] == 'Exempt').any():
        columns_with_exempt.append(col)


print(columns_with_exempt)

['loan_to_value_ratio', 'interest_rate', 'rate_spread', 'total_loan_costs', 'total_points_and_fees', 'origination_charges', 'discount_points', 'lender_credits', 'loan_term', 'prepayment_penalty_term', 'intro_rate_period', 'property_value', 'multifamily_affordable_units', 'debt_to_income_ratio']


In [13]:
# Replacing the word 'Exempt" with NA values and changing the column to numeric

# Iterate through the specified columns
for col in columns_with_exempt:
    # Replace the word 'Exempt' with NA
    df[col] = df[col].replace('Exempt', np.nan)
    
    # Convert the column to a numeric data type
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [14]:
# KNN Imputer does not work with columns that are 100% NA values. 
# Finding these columns and dropping them.
na_percent = df.isna().mean()
df = df.drop(columns = na_percent[na_percent > .99].index.tolist()) 

In [17]:
# With the non-numeric columns and 100% NAs out of the way, we can begin KNN imputation
KNN_imputer = KNNImputer(n_neighbors=4)

#df = df.select_dtypes(include=[np.number])

df = pd.DataFrame(KNN_imputer.fit_transform(df), columns=df.columns)

ValueError: could not convert string to float: 'DRMSV1Q0EKMEXLAU1P80'

#### Train/test split